In [1]:
pip install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
from IPython.display import Markdown, HTML, display

In [3]:
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI
import openai

from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

In [4]:
# Load the API key from the environment variable
api_key = os.getenv('OPENAI_API_KEY')

In [5]:
model = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=api_key 
)


In [6]:
message = HumanMessage(
    content="Translate this sentence from English "
    "to French and Spanish. I like red cars and "
    "blue houses, but my dog is yellow."
)

In [7]:
model.invoke([message])

AIMessage(content="Sure! Here are the translations:\n\n**French:** J'aime les voitures rouges et les maisons bleues, mais mon chien est jaune.\n\n**Spanish:** Me gustan los coches rojos y las casas azules, pero mi perro es amarillo.")

## Load the dataset from CSV

In [8]:
os.makedirs("data", exist_ok=True)

In [9]:
# Run the wget command to download the file and save it to the "data" directory
# Run the curl command to download the file and save it to the "data" directory
!curl -o ./data/all-states-history.csv https://covidtracking.com/data/download/all-states-history.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2674k    0 2674k    0     0  5869k      0 --:--:-- --:--:-- --:--:-- 5864k


In [10]:
file_url = "./data/titanic.csv"
df = pd.read_csv("./data/titanic.csv").fillna(value = 0)

In [11]:
df

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...,...
882,0,2,Rev. Juozas Montvila,male,27.0,0,0,13.0000
883,1,1,Miss. Margaret Edith Graham,female,19.0,0,0,30.0000
884,0,3,Miss. Catherine Helen Johnston,female,7.0,1,2,23.4500
885,1,1,Mr. Karl Howell Behr,male,26.0,0,0,30.0000


In [12]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(llm=model,df=df,verbose=True)

agent.invoke("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: To determine the number of rows in the dataframe `df`, I can use the `shape` attribute of the dataframe, which returns a tuple where the first element is the number of rows and the second element is the number of columns.

Action: Use the `shape` attribute to find the number of rows in the dataframe `df`.
Action Input: df.shape
Observation: Use the `shape` attribute to find the number of rows in the dataframe `df`. is not a valid tool, try one of [python_repl_ast].
Thought:I need to use the Python shell to execute the command and find the number of rows in the dataframe `df`.

Action: Use the `shape` attribute to find the number of rows in the dataframe `df`.
Action Input: df.shape
Observation: Use the `shape` attribute to find the number of rows in the dataframe `df`. is not a valid tool, try one of [python_repl_ast].
Thought:I need to use the Python shell to execute the command and find the number of rows in the dataframe `df`.

Action

{'input': 'how many rows are there?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [13]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

QUESTION = "How many people survived" 
"of Male gender?"
"Use the Survived column" 


agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)



> Entering new AgentExecutor chain...
Thought: To determine how many people survived, I need to count the number of entries in the "Survived" column that have a value of 1. First, I will set the pandas display options to show all columns, then I will get the column names, and finally, I will count the number of survivors.

Action: Set pandas display options to show all columns and get the column names.
Action Input: 
```python
import pandas as pd

# Set display options to show all columns
pd.set_option('display.max_columns', None)

# Get column names
df.columns
```
Observation: Set pandas display options to show all columns and get the column names. is not a valid tool, try one of [python_repl_ast].
Thought:I need to use the Python shell to set the pandas display options and get the column names. Then, I will count the number of survivors in the "Survived" column.

Action: Set pandas display options to show all columns and get the column names.
Action Input: 
```python
import pandas 

{'input': '\nFirst set the pandas display options to show all the columns,\nget the column names, then answer the question.\nHow many people survived\n- **ALWAYS** before giving the Final Answer, try another method.\nThen reflect on the answers of the two methods you did and ask yourself\nif it answers correctly the original question.\nIf you are not sure, try another method.\n- If the methods tried do not give the same result,reflect and\ntry again until you have two methods that have the same result.\n- If you still cannot arrive to a consistent result, say that\nyou are not sure of the answer.\n- If you are sure of the correct answer, create a beautiful\nand thorough response using Markdown.\n- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,\nONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.\n- **ALWAYS**, as part of your "Final Answer", explain how you got\nto the answer on a section that starts with: "\n\nExplanation:\n".\nIn the explanation, mention the column names th

In [13]:
sum(df['Survived'])

342

In [14]:
from IPython.display import Markdown, HTML, display
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain_openai import AzureChatOpenAI

# Connect with SQLite Database

In [15]:
from sqlalchemy import create_engine


In [16]:
import sqlite3

In [17]:
MSSQL_AGENT_PREFIX = """

You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct {dialect} query
to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to
obtain, **ALWAYS** limit your query to at most {top_k} results.
- You can order the results by a relevant column to return the most
interesting examples in the database.
- Never query for all the columns from a specific table, only ask for
the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it.If you get an error
while executing a query,rewrite the query and try again.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.)
to the database.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS
OF THE CALCULATIONS YOU HAVE DONE.
- Your response should be in Markdown. However, **when running  a SQL Query
in "Action Input", do not include the markdown backticks**.
Those are only for formatting the response, not for executing the command.
- ALWAYS, as part of your final answer, explain how you got to the answer
on a section that starts with: "Explanation:". Include the SQL query as
part of the explanation section.
- If the question does not seem related to the database, just return
"I don\'t know" as the answer.
- Only use the below tools. Only use the information returned by the
below tools to construct your query and final answer.
- Do not make up table names, only use the tables returned by any of the
tools below.

## Tools:

"""

In [18]:
MSSQL_AGENT_FORMAT_INSTRUCTIONS = """

## Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: the action to take, should be one of [{tool_names}].
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question.

Example of Final Answer:
<=== Beginning of example

Action: query_sql_db
Action Input: 
SELECT Total
FROM invoices 
WHERE InvoiceDate LIKE '2012%'
LIMIT 10;

Observation:
[477.53]
Thought:I now know the final answer
Final Answer: the total sales amount from the invoices in 2012, considering only the first 10 records is 477.53.

Explanation:
I queried the `invoices` table for the `total` column where the date starts with '2012'. The query returned a single tuples
with the total sales amount from the invoices. To answer the question,
I took the sum, which is 477.53.
I used the following query

```sql
SELECT SUM(Total) AS SUM
FROM invoices 
WHERE InvoiceDate LIKE '2012%'
LIMIT 10;
```
===> End of Example

"""

In [19]:
database_file_path = './data/chinook.db'

In [20]:
db = SQLDatabase.from_uri(f'sqlite:///{database_file_path}')
toolkit = SQLDatabaseToolkit(db=db, llm=model)

In [32]:


from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)
#agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)
agent_executor = create_sql_agent(llm,  
                                  prefix=MSSQL_AGENT_PREFIX,
                                  format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS, 
                                  agent_type="openai-tools", 
                                  top_k = 30, 
                                  verbose=True, 
                                  toolkit = toolkit)

In [33]:
QUESTION = """What is the total sales amount from the invoices in 2012, considering only the first 10 records?
"""
QUESTION2 = """What is the best seller Album?
"""

agent_executor.invoke(
    QUESTION2
)




> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


albums, artists, customers, employees, genres, invoice_items, invoices, media_types, playlist_track, playlists, tracks
Invoking: `sql_db_schema` with `{'table_names': 'albums, invoice_items, tracks'}`



CREATE TABLE albums (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES artists ("ArtistId")
)

/*
3 rows from albums table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE invoice_items (
	"InvoiceLineId" INTEGER NOT NULL, 
	"InvoiceId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	"Quantity" INTEGER NOT NULL, 
	PRIMARY KEY ("InvoiceLineId"), 
	FOREIGN KEY("TrackId") REFERENCES tracks ("TrackId"), 
	FOREIGN KEY("InvoiceId") REFERENCES invoices ("InvoiceId")
)

/*
3 rows from inv

{'input': 'What is the best seller Album?\n',
 'output': 'The best-selling album is "Minha Historia" with a total of 27 sales.\n\nExplanation:\nTo determine the best-selling album, I joined the `albums`, `tracks`, and `invoice_items` tables. I summed the quantity of each track sold for each album and then ordered the results by the total sales in descending order. The query used was:\n\n```sql\nSELECT a.Title, SUM(ii.Quantity) as TotalSales \nFROM albums a \nJOIN tracks t ON a.AlbumId = t.AlbumId \nJOIN invoice_items ii ON t.TrackId = ii.TrackId \nGROUP BY a.Title \nORDER BY TotalSales DESC \nLIMIT 1;\n```'}

In [23]:
agent_executor.invoke(
    "List the total sales per country. Which country's customers spent the most?"
)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


albums, artists, customers, employees, genres, invoice_items, invoices, media_types, playlist_track, playlists, tracks
Invoking: `sql_db_schema` with `{'table_names': 'customers'}`



CREATE TABLE customers (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES employees ("EmployeeId")
)

/*
3 rows from customers table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigadeiro Faria Lima, 2170	São José do

{'input': "List the total sales per country. Which country's customers spent the most?",
 'output': 'The total sales per country are as follows:\n\n1. USA: $523.06\n2. Canada: $303.96\n3. France: $195.10\n4. Brazil: $190.10\n5. Germany: $156.48\n6. United Kingdom: $112.86\n7. Czech Republic: $90.24\n8. Portugal: $77.24\n9. India: $75.26\n10. Chile: $46.62\n\nThe country whose customers spent the most is the USA with total sales of $523.06.'}